In [5]:
import os
import json
from pprint import pprint
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from tqdm.notebook import tqdm

def sv_day(dic):
    runday = list(dic.values())
    run = runday[:-1].count(1)
    sleep = runday[:-1].count(0)
    if run == 8:
        return '每日行駛'
    if run > sleep:
        str = '停'
        special = 0
    else:
        str = '駛'
        special = 1
    for key, word in zip(reversed(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Holiday", "TheDayBeforeHoliday"]), reversed(["一", "二", "三", "四", "五", "六", "日", "及例", "及例前一日"])):
        if dic[key] == special:
            str = word + str
    return str



def create_img(ID):

    with open('TRATimeTable_2022-03-29_0301.json', 'r', encoding='utf8') as f:
        trains = json.load(f)['TrainTimetables']
        for train in trains:
            info = train['GeneralTrainInfo']
            if info['TrainNo'] == ID:
                TYPE = info['TrainTypeName']
                SERVICE = sv_day(info['ServiceDay'])
                STARTSTA = info['ServiceDay']
                VIA = info['ServiceDay']
                startsta = info['StartingStationName'] if info['StartingServiceStationName'] == '' else info['StartingServiceStationName']
                endsta = info['EndingStationName'] if info['EndingServiceStationName'] == '' else info['EndingServiceStationName']
                info_lst = [info['TrainNo']+'次', info['TrainTypeName'],\
                            startsta, '經'+info['TripLine']+'線', '往', endsta,\
                            sv_day(info['ServiceDay'])]
                info_lst = list(filter(lambda x: x not in ['', '經線'], info_lst))
                title = '  '.join(info_lst)+'\n'
                break

    with open(f'.\\Cleaned_old\\{ID}.json', 'r', encoding='utf8') as f: ##############################################################
        json_file = json.load(f)

    y = [i['StationNameZh_tw'] for i in json_file[list(json_file.keys())[0]]]
    x = list(json_file.keys())

    imshow_data = []
    for i in range(len(y)):
        imshow_data += [[json_file[k][i]['DelayTime'] for k in x]]

    fig = px.imshow(imshow_data, x= x, y= y ,color_continuous_scale='RdBu',\
        zmin=-1,\
        zmax=45,\
        title=title,\
        labels= {'color':'誤點時間(分)'},\
        height=800, width=1000)

    fig.update_layout(font_color='#000', font_size=16,margin=dict(l=85,\
                                                                r=50,\
                                                                b=70,\
                                                                t=90,),\
                                                                title_x=0.5,
                                                                title_yanchor='top')

    fig.update_coloraxes( colorscale=[[0, '#c8c8c8'],\
                                    [0.001, '#ffffff'],\
                                    [0.2, '#fddbc7'],\
                                    [0.4, '#f4a582'],\
                                    [0.6, '#d6604d'],\
                                    [0.8, '#b2182b'],\
                                    [0.999, '#67001f'],\
                                    [1, '#4a235a']])

    #fig.update_yaxes(showgrid=True,zeroline=True,showline=True,mirror='all',zerolinecolor='#000000')
    #fig.update_xaxes(tickformat='%m/%d',)

    return fig


In [12]:
fig = create_img('510')
fig2 = create_img('511')
fig3 = create_img('2532')
fig4 = create_img('506')
fig.show()
fig2.show()
fig3.show()
fig4.show()

In [ ]:
route = '.\\Cleaned'
for i in os.walk(route):
    fs = i[2]

ids = [i.replace('.json', '') for i in fs]
ids = ['4194']

In [ ]:
for ID, tq in zip(ids, tqdm(ids)):
    fig = create_img(ID)
    #fig.show()
    #fig.write_html(f'.\\IMAGE_HTML\\{ID}.html')
    fig.write_image(f".\\IMAGE_PNG\\{ID}.pdf",engine='orca')
    fig.write_image(f".\\IMAGE_SVG\\{ID}.svg",engine='orca')

# How to make subplot: https://plotly.com/python/facet-plots/